In [7]:

import geopandas
from shapely.geometry import Point
import numpy as np
import pandas as pd
import io
import re
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
pd.options.display.max_rows = 10

In [12]:
neighborhoods = geopandas.read_file('nynta_17d/nynta.shp')
display(neighborhoods)

,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area,geometry
0,3,Brooklyn,047,BK88,Borough Park,39247.228028,5.400502e+07,"POLYGON ((990897.9000244141 169268.1207885742,..."
1,4,Queens,081,QN52,East Flushing,25843.364936,2.945438e+07,"POLYGON ((1041061.948242188 215429.4609985352,..."
2,4,Queens,081,QN48,Auburndale,32446.878673,3.416422e+07,"POLYGON ((1046392.971191406 210169.7536010742,..."
3,4,Queens,081,QN51,Murray Hill,33266.904811,5.248828e+07,"POLYGON ((1038593.459228516 221913.3550415039,..."
4,4,Queens,081,QN27,East Elmhurst,19816.712323,1.972685e+07,"POLYGON ((1022728.275024414 217530.8082275391,..."
...,...,...,...,...,...,...,...,...
190,1,Manhattan,061,MN23,West Village,29400.994067,2.500482e+07,"POLYGON ((987048.2695922852 206851.1467895508,..."
191,2,Bronx,005,BX09,Soundview-Castle Hill-Clason Point-Harding Park,67341.077044,5.198380e+07,(POLYGON ((1017288.198608398 237508.7247924805...
192,2,Bronx,005,BX99,park-cemetery-etc-Bronx,223443.788237,2.208418e+08,(POLYGON ((1020356.162841797 236653.0676269531...
193,4,Queens,081,QN28,Jackson Heights,29422.388817,4.785688e+07,"POLYGON ((1019252.57019043 215025.8591918945, ..."
